In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import shap
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import LabelEncoder 

In [ ]:
df = pd.read_csv('/kaggle/input/fast-food/FastFoodNutritionMenu.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
unique_values =  df.nunique()
unique_values

In [ ]:
df['Company'].value_counts()

In [ ]:
cols = df.columns
cols

# transform a datatype

In [ ]:
columns = ['Calories','Calories from\nFat','Total Fat\n(g)','Saturated Fat\n(g)','Trans Fat\n(g)','Cholesterol\n(mg)','Sodium \n(mg)',
           'Carbs\n(g)','Fiber\n(g)','Sugars\n(g)','Protein\n(g)','Weight Watchers\nPnts']
for column in columns :
    if df[column].dtype == object:  # Check if the column contains strings
        df[column] = pd.to_numeric(df[column].str.replace('\xa0', ''), errors='coerce')
    
df.dtypes

# Null vales 

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().plot(kind = 'bar')

In [ ]:
# Drop the 'Weight Watchers\nPnts' column
df.drop(columns=['Weight Watchers\nPnts'] )
# Fill missing values in 'Protein\n(g)' with the mean
df['Protein\n(g)'] = df['Protein\n(g)'].fillna(df['Protein\n(g)'].mean())
# Drop rows with any remaining missing values
df = df.dropna()
# Check for missing values 
df.isna().sum()

In [ ]:
df.shape

# Finding duplicate rows


In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
# Remove duplicates and update the DataFrame
df.drop_duplicates(keep='first', inplace=True)

# # Reduce Unnecessary Columns


In [ ]:
plt.figure(figsize=(25,25))
Corr_Matrix=df.corr()
sns.heatmap(df.corr(), annot=True, fmt=".2f")
plt.show()

In [ ]:
print('Top 5 Most Positively Correlated to the Target Variable')
Corr_Matrix['Calories'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Negatively Correlated to the Target Variable')
Corr_Matrix['Calories'].sort_values(ascending=True).head(5)

# visualization 

In [ ]:
top_10_Value_Calories = df.nlargest(10, 'Calories')
top_10_Value_Calories

In [ ]:
fig1 = px.bar(top_10_Value_Calories, x='Calories', y='Item', orientation='h', text='Calories',
              color='Item', labels={'Calories': 'Calories (in Item)'},
              color_continuous_scale='Viridis')  
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside')
fig1.update_layout(title_text='Top 10 Calories value in Item', yaxis_title='Calories',
                   xaxis_title='Calories (in Item)', height=500)
fig1.show()

In [ ]:
df[df['Calories']>1000.0]["Calories from\nFat"].plot(kind="bar")

In [ ]:

# Sort the data based on the "Price" column from low to high
data_sorted = df.sort_values(by='Calories')

# Get a list of all columns except for 'Price'
columns_to_graph = df.columns.tolist()
columns_to_graph= ['Total Fat\n(g)','Saturated Fat\n(g)', 'Trans Fat\n(g)','Weight Watchers\nPnts']
# Plot each column against 'Price' for the sorted data

for column in columns_to_graph:
    plt.figure(figsize=(12, 8))  # Set the figure size
    plt.scatter(data_sorted[column], data_sorted['Calories'])
    plt.xlabel(column)
    plt.ylabel('Calories')
    plt.xticks(rotation='vertical')  # Rotate x-axis labels vertically
    plt.show()

In [ ]:
sns.violinplot(x=df["Calories"])

In [ ]:
sns.distplot(x =df['Calories'])

In [ ]:
columns_to_plot = ['Calories','Calories from\nFat','Total Fat\n(g)','Saturated Fat\n(g)','Cholesterol\n(mg)','Sodium \n(mg)',
           'Carbs\n(g)','Fiber\n(g)','Protein\n(g)','Weight Watchers\nPnts']
for column in columns_to_plot:
    plt.figure(figsize=(8, 6))
    plt.boxplot(df[column], vert=False)
    plt.title(f'Box Plot of {column}')
    plt.xlabel(column)
    plt.show()

In [ ]:
filtered_df = df[(df['Company'] == 'Burger King')&(df['Calories']>650)]

In [ ]:
fig = px.bar(filtered_df, x='Item', y='Calories')#create relation 
# Updates graph layout/styles
fig.update_layout(title={'text': 'Item Calories ','x': 0.5,})#for the title
# Shows the graph
fig.show()

In [ ]:
filtered_df1 = df[(df['Item'].str.contains('Double')) & (df['Calories'] > 650)]

In [ ]:
fig = px.bar(filtered_df1, x='Item', y='Calories')#create relation 
# Updates graph layout/styles
fig.update_layout(title={'text': 'Item Calories','x': 0.5,})#for the title
# Shows the graph
fig.show()

In [ ]:
import plotly.express as px
companies = df['Company'].unique()
# Define nutritional columns
nutritional_columns = ['Total Fat\n(g)', 'Protein\n(g)', 'Carbs\n(g)']

# Create box plots for each nutritional column
fig = px.box(df, x='Company', y=nutritional_columns, title='Nutritional Values by Company',
             labels={'value': 'Grams'}, category_orders={'Company': companies})
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': companies},
                  xaxis_title='Company', yaxis_title='Grams')

fig.show()

In [ ]:
sns.barplot(x=df['Company'], y=df['Total Fat\n(g)'])

# categorical

In [ ]:
le = LabelEncoder()

In [ ]:
df['Company'] = le.fit_transform(df['Company'])
df['Item'] = le.fit_transform(df['Item'])

# Model

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
X = df.drop(columns=['Calories'])
y = df['Calories']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual CO2'] = y_test
    submit['Predict_CO2'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")

In [ ]:
importances = model.feature_importances_

feature_names = X.columns

feature_importance_dict = dict(zip(feature_names, importances))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.2f}")

plt.figure(figsize=(12, 7))
plt.barh(*zip(*sorted_feature_importance), alpha=0.9, color='teal')
plt.title('Feature Importance', fontsize=15)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
import statsmodels.api as sm

X = df.drop(columns=['Calories'])
y = df['Calories']

def forward_selection(df, target, significance_level=0.05):
    initial_features = df.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(df[best_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

# Assuming you have already defined X and y as the features and target variable respectively
selected_features = forward_selection(X, y)
print("Selected features:", selected_features)

In [ ]:
Selected_features = ['Weight Watchers\nPnts', 'Sugars\n(g)', 'Protein\n(g)', 'Saturated Fat\n(g)', 'Item']
X = df[Selected_features]
y = df['Calories']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual CO2'] = y_test
    submit['Predict_CO2'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")